## Features

In [ ]:
import pandas as pd
import numpy as np


# 1. Flag de campanha anterior
train['foi_contatado_antes'] = (train['pdays'] != -1).astype(int)

# 2. Meses de alta conversão (seasonality)
high_season_months = ['mar', 'sep', 'oct', 'dec']
train['is_high_season'] = train['month'].isin(high_season_months).astype(int)

# 3. Bins de balance
train['balance_category'] = pd.cut(train['balance'], 
                                 bins=[-np.inf, 0, 1000, 5000, np.inf],
                                 labels=['negative', 'low', 'medium', 'high'])

# 4. Bins de age
train['age_group'] = pd.cut(train['age'], 
                          bins=[0, 25, 35, 50, 65, 100],
                          labels=['jovem', 'adulto_jovem', 'adulto', 'senior', 'aposentado'])

# 5. Intensidade de campanha (normalizado)
train['campaign_intensity'] = train['campaign'] + train['previous']

# 6. Taxa de sucesso histórico (binária)
train['historico_positivo'] = (train['poutcome'] == 'success').astype(int)

# 7. Profissões de alta conversão
high_conversion_jobs = ['student', 'retired', 'unemployed', 'management']
train['job_high_conversion'] = train['job'].isin(high_conversion_jobs).astype(int)

# 8. Tem contato efetivo (não é unknown)
train['has_effective_contact'] = (train['contact'] != 'unknown').astype(int)

# 9. Razão balance/age (capacidade financeira relativa)
train['balance_per_age'] = train['balance'] / (train['age'] + 1)

# 10. Flag de cliente "frio" (muitos contatos sem sucesso)
train['cliente_frio'] = ((train['campaign'] > 3) & (train['poutcome'] != 'success')).astype(int)


# Ajustando Features

## Arquivos

In [1]:
import pandas as pd

TRAIN_DIR = "../data/raw/analise-preditiva-de-comportamento-bancario/train.csv"
TEST_DIR = "../data/raw/analise-preditiva-de-comportamento-bancario/test.csv"
SAMPLE_DIR = "../data/raw/analise-preditiva-de-comportamento-bancario/sample_submission.csv"

train = pd.read_csv(TRAIN_DIR)
test = pd.read_csv(TEST_DIR)
sample = pd.read_csv(SAMPLE_DIR)

#print(f"Train shape: {train.shape}")
#print(f"Test shape: {test.shape}")

# Separar target
y_train = train['y']
X_train = train.drop('y', axis=1)

# Concatenar para aplicar transformações uniformemente
df_combined = pd.concat([X_train, test], axis=0, ignore_index=True)
print(f"\nCombined shape: {df_combined.shape}")


Combined shape: (1000000, 17)


## Features Binárias

In [2]:
def create_binary_features(df):
    """Cria features binárias baseadas em insights do EDA"""
    
    df = df.copy()
    
    # 1.1 Foi contatado anteriormente
    df['foi_contatado_antes'] = (df['pdays'] != -1).astype(int)
    
    # 1.2 Histórico de sucesso em campanhas
    df['historico_sucesso'] = (df['poutcome'] == 'success').astype(int)
    df['historico_failure'] = (df['poutcome'] == 'failure').astype(int)
    df['historico_unknown'] = (df['poutcome'] == 'unknown').astype(int)
    
    # 1.3 Profissões de alta conversão (>15%)
    high_conversion_jobs = ['student', 'retired', 'unemployed', 'management']
    df['job_high_conversion'] = df['job'].isin(high_conversion_jobs).astype(int)
    
    # 1.4 Contato efetivo (não é unknown)
    df['has_effective_contact'] = (df['contact'] != 'unknown').astype(int)
    df['contact_cellular'] = (df['contact'] == 'cellular').astype(int)
    
    # 1.5 Situação financeira
    df['has_default'] = (df['default'] == 'yes').astype(int)
    df['has_housing'] = (df['housing'] == 'yes').astype(int)
    df['has_loan'] = (df['loan'] == 'yes').astype(int)
    df['no_debt'] = ((df['default'] == 'no') & (df['loan'] == 'no')).astype(int)
    
    # 1.6 Status civil
    df['is_single'] = (df['marital'] == 'single').astype(int)
    df['is_married'] = (df['marital'] == 'married').astype(int)
    
    # 1.7 Educação superior
    df['high_education'] = (df['education'].isin(['tertiary', 'unknown'])).astype(int)
    
    return df

df_combined = create_binary_features(df_combined)
print("Features binárias criadas:")
binary_features = [col for col in df_combined.columns if col.startswith(('foi_', 'historico_', 'job_', 'has_', 'is_', 'contact_', 'no_', 'high_'))]
print(binary_features)


Features binárias criadas:
['foi_contatado_antes', 'historico_sucesso', 'historico_failure', 'historico_unknown', 'job_high_conversion', 'has_effective_contact', 'contact_cellular', 'has_default', 'has_housing', 'has_loan', 'no_debt', 'is_single', 'is_married', 'high_education']


## Features Temporais

In [3]:
def create_temporal_features(df):
    """Cria features baseadas em padrões temporais"""
    
    df = df.copy()
    
    # 2.1 Meses de alta conversão (>40% conversão no EDA)
    high_season_months = ['mar', 'sep', 'oct', 'dec']
    df['is_high_season'] = df['month'].isin(high_season_months).astype(int)
    
    # 2.2 Mês com pior conversão
    df['is_may'] = (df['month'] == 'may').astype(int)
    
    # 2.3 Trimestres
    month_to_quarter = {
        'jan': 1, 'feb': 1, 'mar': 1,
        'apr': 2, 'may': 2, 'jun': 2,
        'jul': 3, 'aug': 3, 'sep': 3,
        'oct': 4, 'nov': 4, 'dec': 4
    }
    df['quarter'] = df['month'].map(month_to_quarter)
    
    # 2.4 Início/meio/fim do mês
    df['day_period'] = pd.cut(df['day'], 
                               bins=[0, 10, 20, 31], 
                               labels=[0, 1, 2])  # 0=início, 1=meio, 2=fim
    df['day_period'] = df['day_period'].astype(int)
    
    # 2.5 Início do mês (dias 1-5)
    df['inicio_mes'] = (df['day'] <= 5).astype(int)
    
    # 2.6 Fim do mês (dias 25-31)
    df['fim_mes'] = (df['day'] >= 25).astype(int)
    
    return df

df_combined = create_temporal_features(df_combined)
print("\nFeatures temporais criadas:")
temporal_features = ['is_high_season', 'is_may', 'quarter', 'day_period', 'inicio_mes', 'fim_mes']
print(temporal_features)



Features temporais criadas:
['is_high_season', 'is_may', 'quarter', 'day_period', 'inicio_mes', 'fim_mes']


## Features de Campanha

In [4]:
def create_campaign_features(df):
    """Cria features relacionadas ao histórico de campanhas"""
    
    df = df.copy()
    
    # 3.1 Total de contatos (atual + anteriores)
    df['total_contacts'] = df['campaign'] + df['previous']
    
    # 3.2 Frequência de contatos (normalizado)
    df['contact_frequency'] = df['campaign'] / (df['campaign'].max() + 1)
    
    # 3.3 Cliente "quente" vs "frio"
    # Frio: muitos contatos sem sucesso
    df['cliente_frio'] = ((df['campaign'] > 3) & (df['poutcome'] != 'success')).astype(int)
    
    # Quente: poucos contatos com histórico positivo
    df['cliente_quente'] = ((df['campaign'] <= 2) & (df['poutcome'] == 'success')).astype(int)
    
    # 3.4 Cliente virgem (sem histórico)
    df['cliente_novo'] = ((df['previous'] == 0) & (df['poutcome'] == 'unknown')).astype(int)
    
    # 3.5 Tempo desde último contato (categorizado)
    # -1 = nunca contatado, 0-30 = recente, 31-180 = médio, >180 = antigo
    df['pdays_category'] = pd.cut(df['pdays'], 
                                    bins=[-2, -1, 30, 180, 999],
                                    labels=[0, 1, 2, 3])  # 0=nunca, 1=recente, 2=médio, 3=antigo
    df['pdays_category'] = df['pdays_category'].astype(int)
    
    # 3.6 Recência do contato (se foi contatado)
    df['contato_recente'] = ((df['pdays'] > 0) & (df['pdays'] <= 30)).astype(int)
    
    # 3.7 Intensidade de campanha
    df['high_campaign_intensity'] = (df['campaign'] > df['campaign'].median()).astype(int)
    
    return df

df_combined = create_campaign_features(df_combined)
print("\nFeatures de campanha criadas:")
campaign_features = ['total_contacts', 'contact_frequency', 'cliente_frio', 'cliente_quente', 
                     'cliente_novo', 'pdays_category', 'contato_recente', 'high_campaign_intensity']
print(campaign_features)



Features de campanha criadas:
['total_contacts', 'contact_frequency', 'cliente_frio', 'cliente_quente', 'cliente_novo', 'pdays_category', 'contato_recente', 'high_campaign_intensity']


## Features Financeiras

In [5]:
import numpy as np

def create_financial_features(df):
    """Cria features baseadas em variáveis financeiras"""
    
    df = df.copy()
    
    # 4.1 Categorização de balance
    df['balance_category'] = pd.cut(df['balance'], 
                                     bins=[-np.inf, 0, 1000, 5000, np.inf],
                                     labels=[0, 1, 2, 3])  # 0=negativo, 1=baixo, 2=médio, 3=alto
    df['balance_category'] = df['balance_category'].astype(int)
    
    # 4.2 Flags de balance
    df['balance_negative'] = (df['balance'] < 0).astype(int)
    df['balance_high'] = (df['balance'] > 5000).astype(int)
    df['balance_zero'] = (df['balance'] == 0).astype(int)
    
    # 4.3 Balance normalizado por idade (capacidade financeira relativa)
    df['balance_per_age'] = df['balance'] / (df['age'] + 1)
    
    # 4.4 Log de balance (para tratar outliers e distribuição)
    # Adicionar constante para lidar com valores negativos
    df['balance_log'] = np.log1p(df['balance'] + abs(df['balance'].min()) + 1)
    
    # 4.5 Situação de endividamento geral
    df['total_loans'] = df['has_housing'] + df['has_loan'] + df['has_default']
    df['sem_dividas'] = (df['total_loans'] == 0).astype(int)
    
    # 4.6 Perfil financeiro composto
    # Alto balance + sem dívidas = perfil "premium"
    df['perfil_premium'] = ((df['balance'] > df['balance'].median()) & 
                            (df['sem_dividas'] == 1)).astype(int)
    
    return df

df_combined = create_financial_features(df_combined)
print("\nFeatures financeiras criadas:")
financial_features = ['balance_category', 'balance_negative', 'balance_high', 'balance_zero',
                      'balance_per_age', 'balance_log', 'total_loans', 'sem_dividas', 'perfil_premium']
print(financial_features)



Features financeiras criadas:
['balance_category', 'balance_negative', 'balance_high', 'balance_zero', 'balance_per_age', 'balance_log', 'total_loans', 'sem_dividas', 'perfil_premium']


## Features Demográficas

In [6]:
def create_demographic_features(df):
    """Cria features demográficas"""
    
    df = df.copy()
    
    # 5.1 Faixas etárias
    df['age_group'] = pd.cut(df['age'], 
                              bins=[0, 25, 35, 50, 65, 100],
                              labels=[0, 1, 2, 3, 4])  # jovem, adulto_jovem, adulto, senior, idoso
    df['age_group'] = df['age_group'].astype(int)
    
    # 5.2 Flags de idade
    df['is_young'] = (df['age'] < 30).astype(int)
    df['is_senior'] = (df['age'] >= 60).astype(int)
    df['working_age'] = ((df['age'] >= 25) & (df['age'] < 65)).astype(int)
    
    # 5.3 Perfis combinados (idade + profissão)
    df['estudante_jovem'] = ((df['job'] == 'student') & (df['age'] < 30)).astype(int)
    df['aposentado_senior'] = ((df['job'] == 'retired') & (df['age'] >= 60)).astype(int)
    
    # 5.4 Perfil socioeconômico (educação + profissão + balance)
    high_status_jobs = ['management', 'self-employed', 'entrepreneur']
    df['high_status'] = ((df['job'].isin(high_status_jobs)) & 
                         (df['education'] == 'tertiary')).astype(int)
    
    # 5.5 Potencial de poupança (balance alto + idade trabalhadora)
    df['poupador_potencial'] = ((df['balance'] > df['balance'].median()) & 
                                 (df['working_age'] == 1)).astype(int)
    
    return df

df_combined = create_demographic_features(df_combined)
print("\nFeatures demográficas criadas:")
demographic_features = ['age_group', 'is_young', 'is_senior', 'working_age',
                        'estudante_jovem', 'aposentado_senior', 'high_status', 'poupador_potencial']
print(demographic_features)



Features demográficas criadas:
['age_group', 'is_young', 'is_senior', 'working_age', 'estudante_jovem', 'aposentado_senior', 'high_status', 'poupador_potencial']


## Features de Interação

In [7]:
def create_interaction_features(df):
    """Cria features de interação entre variáveis"""
    
    df = df.copy()
    
    # 6.1 Idade x Balance (poder aquisitivo por idade)
    df['age_balance_ratio'] = (df['age'] * df['balance']) / 1000
    
    # 6.2 Campanha x Histórico (efetividade de recontato)
    df['campaign_success_interaction'] = df['campaign'] * df['historico_sucesso']
    
    # 6.3 Educação x Profissão (consistência de perfil)
    df['education_job_match'] = ((df['education'] == 'tertiary') & 
                                  (df['job'].isin(['management', 'technician', 'services']))).astype(int)
    
    # 6.4 Contato x Mês (timing de contato)
    df['contact_timing'] = df['contact_cellular'] * df['is_high_season']
    
    # 6.5 Balance x Empréstimos (risco financeiro)
    df['balance_loan_ratio'] = df['balance'] / (df['total_loans'] + 1)
    
    # 6.6 Previous x Poutcome (histórico ponderado)
    poutcome_weight = {'success': 3, 'failure': -1, 'other': 0, 'unknown': 0}
    df['poutcome_numeric'] = df['poutcome'].map(poutcome_weight)
    df['weighted_history'] = df['previous'] * df['poutcome_numeric']
    
    return df

df_combined = create_interaction_features(df_combined)
print("\nFeatures de interação criadas:")
interaction_features = ['age_balance_ratio', 'campaign_success_interaction', 'education_job_match',
                        'contact_timing', 'balance_loan_ratio', 'weighted_history']
print(interaction_features)



Features de interação criadas:
['age_balance_ratio', 'campaign_success_interaction', 'education_job_match', 'contact_timing', 'balance_loan_ratio', 'weighted_history']


## Encoding de Variáveis Categóricas

In [8]:
def encode_categorical_features(df, target=None, train_size=None):
    """
    Aplica encoding nas variáveis categóricas
    - Target Encoding para alta cardinalidade
    - Label Encoding para ordinais
    - One-Hot para baixa cardinalidade
    """
    
    df = df.copy()
    
    # 7.1 Label Encoding para month (ordem temporal)
    month_order = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
                   'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}
    df['month_encoded'] = df['month'].map(month_order)
    
    # 7.2 Target Encoding para job e education (alta cardinalidade)
    # Apenas se tivermos o target (durante treino)
    if target is not None and train_size is not None:
        # Calcular médias no conjunto de treino
        df_train = df.iloc[:train_size].copy()
        df_train['target'] = target
        
        # Job
        job_means = df_train.groupby('job')['target'].mean()
        df['job_encoded'] = df['job'].map(job_means)
        df['job_encoded'].fillna(job_means.mean(), inplace=True)
        
        # Education
        education_means = df_train.groupby('education')['target'].mean()
        df['education_encoded'] = df['education'].map(education_means)
        df['education_encoded'].fillna(education_means.mean(), inplace=True)
        
    else:
        # Se não tiver target, usar label encoding simples
        le_job = LabelEncoder()
        le_education = LabelEncoder()
        df['job_encoded'] = le_job.fit_transform(df['job'])
        df['education_encoded'] = le_education.fit_transform(df['education'])
    
    # 7.3 One-Hot Encoding para variáveis de baixa cardinalidade
    # Marital
    df['marital_single'] = (df['marital'] == 'single').astype(int)
    df['marital_married'] = (df['marital'] == 'married').astype(int)
    df['marital_divorced'] = (df['marital'] == 'divorced').astype(int)
    
    # Contact (já fizemos algumas flags, completar)
    df['contact_telephone'] = (df['contact'] == 'telephone').astype(int)
    df['contact_unknown'] = (df['contact'] == 'unknown').astype(int)
    
    # Poutcome (já temos flags, adicionar other)
    df['poutcome_other'] = (df['poutcome'] == 'other').astype(int)
    
    return df

# Aplicar encoding
df_combined = encode_categorical_features(df_combined, target=y_train, train_size=len(X_train))
print("\nEncoding aplicado às variáveis categóricas")


C:\Users\emaci\AppData\Local\Temp\ipykernel_49924\1372905346.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['job_encoded'].fillna(job_means.mean(), inplace=True)
C:\Users\emaci\AppData\Local\Temp\ipykernel_49924\1372905346.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy


Encoding aplicado às variáveis categóricas


## Normalização (depende do modelo)

In [9]:
# para modelos lineares
from sklearn.preprocessing import StandardScaler

def normalize_features(df, train_size, features_to_normalize=None):
    """
    Normaliza features numéricas usando StandardScaler
    Ajusta no train e transforma train+test
    """
    
    df = df.copy()
    
    if features_to_normalize is None:
        # Features numéricas que se beneficiam de normalização
        features_to_normalize = [
            'age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
            'balance_per_age', 'age_balance_ratio', 'balance_loan_ratio'
        ]
    
    scaler = StandardScaler()
    
    # Fit apenas no train
    scaler.fit(df.iloc[:train_size][features_to_normalize])
    
    # Transform em train+test
    df[features_to_normalize] = scaler.transform(df[features_to_normalize])
    
    return df, scaler

# Guardar versão sem normalização (para tree-based models)
df_combined_original = df_combined.copy()

# Versão normalizada (para modelos lineares, se necessário)
# df_combined_normalized, scaler = normalize_features(df_combined, len(X_train))
# print("\nNormalização aplicada (versão separada criada)")


## Seleção de Features e Preparação Final

In [10]:
# Features originais a remover (já foram transformadas)
features_to_drop = ['job', 'marital', 'education', 'default', 'housing', 'loan', 
                    'contact', 'month', 'poutcome', 'poutcome_numeric']

# Remover features originais categóricas
df_final = df_combined.drop(columns=features_to_drop, errors='ignore')

print(f"\nShape após feature engineering: {df_final.shape}")
print(f"Número de features criadas: {df_final.shape[1] - len(X_train.columns) + len(features_to_drop)}")

# Separar novamente em train e test
X_train_processed = df_final.iloc[:len(X_train)].copy()
X_test_processed = df_final.iloc[len(X_train):].copy()

# Adicionar o target de volta ao train
X_train_processed['y'] = y_train.values

print(f"\nTrain processed shape: {X_train_processed.shape}")
print(f"Test processed shape: {X_test_processed.shape}")

# Verificar valores faltantes
print(f"\nValores faltantes no train: {X_train_processed.isnull().sum().sum()}")
print(f"Valores faltantes no test: {X_test_processed.isnull().sum().sum()}")



Shape após feature engineering: (1000000, 68)
Número de features criadas: 61

Train processed shape: (750000, 69)
Test processed shape: (250000, 68)

Valores faltantes no train: 0
Valores faltantes no test: 0


## Salvar Dados Processados

In [11]:
# Salvar datasets processados
X_train_processed.to_csv('../data/processed/train_processed.csv', index=False)
X_test_processed.to_csv('../data/processed/test_processed.csv', index=False)

print("\nDados processados salvos em data/processed/")
print("\nArquivos criados:")
print("- train_processed.csv")
print("- test_processed.csv")

# Mostrar lista de todas as features criadas
all_features = X_train_processed.drop('y', axis=1).columns.tolist()
print(f"\nTotal de features: {len(all_features)}")
print("\nPrimeiras 20 features:")
print(all_features[:20])


Dados processados salvos em data/processed/

Arquivos criados:
- train_processed.csv
- test_processed.csv

Total de features: 68

Primeiras 20 features:
['id', 'age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'foi_contatado_antes', 'historico_sucesso', 'historico_failure', 'historico_unknown', 'job_high_conversion', 'has_effective_contact', 'contact_cellular', 'has_default', 'has_housing', 'has_loan', 'no_debt', 'is_single']


## Resumo

In [12]:
print("\n" + "="*70)
print("RESUMO DA FEATURE ENGINEERING")
print("="*70)

print("\nFeatures Criadas por Categoria:")
print(f"  • Binárias (flags): {len(binary_features)}")
print(f"  • Temporais: {len(temporal_features)}")
print(f"  • Campanha: {len(campaign_features)}")
print(f"  • Financeiras: {len(financial_features)}")
print(f"  • Demográficas: {len(demographic_features)}")
print(f"  • Interações: {len(interaction_features)}")

print(f"\nTotal de features: {X_train_processed.shape[1] - 1} (excluindo target)")
print(f"Features originais: {X_train.shape[1]}")
print(f"Features novas: {X_train_processed.shape[1] - 1 - X_train.shape[1]}")

print("\nDecisões Importantes:")
print("  • Duration: MANTIDA")
print("  • Target Encoding: Aplicado em job e education")
print("  • Normalização: Preparada mas não aplicada (usar para Linear Models)")
print("  • Dados salvos: ../data/processed/")


RESUMO DA FEATURE ENGINEERING

Features Criadas por Categoria:
  • Binárias (flags): 14
  • Temporais: 6
  • Campanha: 8
  • Financeiras: 9
  • Demográficas: 8
  • Interações: 6

Total de features: 68 (excluindo target)
Features originais: 17
Features novas: 51

Decisões Importantes:
  • Duration: MANTIDA
  • Target Encoding: Aplicado em job e education
  • Normalização: Preparada mas não aplicada (usar para Linear Models)
  • Dados salvos: ../data/processed/
